# CS483 - Colab 2
## Frequent Pattern Mining in Spark

In [ ]:
from IPython.display import Image
print("Colab 2 Mascot")
Image(url='https://cdn.dribbble.com/users/222579/screenshots/1654898/stubby-ben-rex-roll.gif',width=150)

Colab 2 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u422-b05-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-7-03ad07eeace4>:5 

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [ ]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [ ]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [ ]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [ ]:
''' 2 lines of code expected '''
# YOUR CODE HERE
order_products = orders.join(products, orders.product_id == products.product_id).select(orders.order_id, products.product_name, orders.add_to_cart_order, orders.reordered)
order_products.show()
baskets = order_products.groupBy('order_id').agg(collect_list('product_name').alias('products')).sort('order_id')
baskets.show()

+--------+--------------------+-----------------+---------+
|order_id|        product_name|add_to_cart_order|reordered|
+--------+--------------------+-----------------+---------+
|       1|    Bulgarian Yogurt|                1|        1|
|       1|Organic 4% Milk F...|                2|        1|
|       1|Organic Celery He...|                3|        0|
|       1|      Cucumber Kirby|                4|        0|
|       1|Lightly Smoked Sa...|                5|        1|
|       1|Bag of Organic Ba...|                6|        0|
|       1|Organic Hass Avocado|                7|        0|
|       1|Organic Whole Str...|                8|        1|
|      36|Grated Pecorino R...|                1|        0|
|      36|        Spring Water|                2|        1|
|      36| Organic Half & Half|                3|        0|
|      36|  Super Greens Salad|                4|        1|
|      36|Cage Free Extra L...|                5|        1|
|      36|Prosciutto, Ameri...|         

In [ ]:
type(baskets)

pyspark.sql.dataframe.DataFrame

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [ ]:
''' 3 lines of code expected '''
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(baskets)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
itemsets = model.freqItemsets
print("Number of frequent itemsets: " + str(itemsets.count()))
itemsets.orderBy(desc("freq")).show()
rules = model.associationRules
print("Number of association rules: " + str(rules.count()))
rules.orderBy(desc("confidence")).show()

Number of frequent itemsets: 120
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

Number of association rules: 0
+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(baskets)

itemsets = model.freqItemsets
print("Number of frequent itemsets: " + str(itemsets.count()))
itemsets.orderBy(desc("freq")).show()

rules = model.associationRules
print("Number of association rules: " + str(rules.count()))
rules.orderBy(desc("confidence")).show()

Number of frequent itemsets: 4444
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

Number of association rules: 11
+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+-----

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [ ]:
# YOUR CODE HERE

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!